# Basic DEA
## Description
Estimate the efficiency of US airlines in 2008. We have two inputs and two outputs as follows. \
Inputs Fuel(FU) is the number of gallons consumed annually,estimated by fuel expenses over the average jet fuel cost per gallon. \
Employee (EP) is defined as the number of employees during the year, which includes flight shipping staff, pilots, flight attendants, and managers butnotground shipping drivers. \
Average prices are calculated by salaries and benefits expenses over number of employees.\
Outputs Passenger Output (PO) is the actual output of available seat-miles during the year. \
Available seat-miles is calculated asthe number of seats including first class andeconomy on anairplane multiplied by the distance traveled measured in miles. \
The average price per passenger mile is calculated as the scheduled passenger revenue divided by passenger-miles.
Freight Output (FO) is the actual output of available freight-ton-miles during the year. \
Available freight-ton-miles is calculated as the number of available tons of freight and mail multiplied by the distance flown measured in miles. \
The average price is calculated as the scheduled freight and mail revenue divided by ton-miles.

In [15]:
from gurobipy import *
import pandas as pd
import numpy as np

### i. (10%) Please define the set, parameters, and decision variables used in input-oriented CRS and VRS formulations of DEA.

**Sets**<br />
$ K=\{1,...,12\} $: 2008 U.S. Airlines as DMUs.<br />
$ I=\{1,2\} $: Types of inputs.<br />
$ J=\{1,2\} $: Types of outputs.<br />
**Parameters**<br />
$ X_{ki} $ Input ${i}$ of DMU $k$.  (<br />
$ Y_{kj} $ Output ${j}$ of DMU $k$.<br />
e.g. <br />
$ X_{k1} $ Gallons of fuel of DMU $k$.<br />
$ X_{k2} $ Units of employee DMU $k$.<br />
$ Y_{k1} $ Passenger miles of DMU $k$.<br />
$ Y_{k2} $ Ton-miles of DMU $k$.<br />
**Decision variables**<br />
$ \theta_{r} $ efficiency of DMU $r$.<br />
$ \lambda_{k} $ coefficients for DMU $k$ of the convex combination for optimization problem.<br />

In [16]:
df = pd.read_csv("Data.csv")

In [17]:
df["Gallons "] = df["Gallons "] *1e6
df["Passenger-miles "] = df["Passenger-miles "] *1e6
df["Ton-miles"] = df["Ton-miles"] *1e6

In [18]:
df.head()

,DMU,No.,Gallons,Prce Per Gallons,Units,Price per unit,Passenger-miles,Price per Passenger-miles,Ton-miles,Price per Ton-miles
0,AirTran Airways,A,3.920000e+08,3.05,8259,57500.9,2.375600e+10,0.13,8.700000e+08,0.56
1,Alaska Airlines,B,3.810000e+08,3.05,9628,78780.6,2.418300e+10,0.14,1.359000e+09,1.76
2,American Airlines,C,2.673000e+09,3.05,70923,85219.2,1.634830e+11,0.14,1.244900e+10,0.43
3,American Eagle Airlines,D,2.820000e+08,3.05,9683,85219.2,1.037000e+10,0.30,5.090000e+08,0.43
4,Continental,E,1.608000e+09,3.05,40630,70145.2,9.904700e+10,0.14,3.726000e+09,0.47


In [19]:
X = [
    list(df["Gallons "].values),
    list(df["Units"].values)
]
X = np.array(X).T.tolist()

Y = [
    list(df["Passenger-miles "].values),
    list(df["Ton-miles"].values)
]
Y = np.array(Y).T.tolist()

# Sets
K = list(range(len(df["DMU"])))
I = [0,1]
J = [0,1]

### ii.(15%) Estimate input-oriented efficiency of each firm. Please show the results of technical efficiency (VRS model), overall efficiency (CRS model), scale efficiency.

$$ 
\begin{array}{r}
\min \limits_{\lambda} \theta_r \\
\text{s.t.}  \\
\end{array}
$$

**input-oriented CRS formulation** <br />
$$ 
\begin{split}
\min \limits_{\lambda} \quad & \theta_r \\
\text{s.t.}  \quad
& \sum_{k \in K} \lambda_k X_{ki} \leq \theta_r X_{ri}  \quad \forall i \in I \\
& \sum_{k \in K} \lambda_k Y_{kj} \geq  Y_{rj}  \quad \forall j \in J \\
& \lambda_k \geq 0 \quad \forall k \in K \\
& \theta_r \text{ is free}.
\end{split}
$$

**input-oriented VRS formulation** <br />
$$ 
\begin{split}
\min \limits_{\lambda} \quad & \theta_r \\
\text{s.t.} \quad
& \sum_{k \in K} \lambda_k X_{ki} \leq \theta_r X_{ri}  \quad \forall i \in I \\
& \sum_{k \in K} \lambda_k Y_{kj} \geq  Y_{rj}  \quad \forall j \in J \\
& \sum_{k \in K} \lambda_k = 1 \\
& \lambda_k \geq 0 \quad \forall k \in K \\
& \theta_r \text{ is free}.
\end{split}
$$

In [20]:
crs_efficiency = [0 for r in K]

for r in K:
    md11 = Model('model1-1')
    l = md11.addVars(K, vtype=GRB.CONTINUOUS, lb=0, name='l')
    t = md11.addVar(vtype=GRB.CONTINUOUS, name='t')
    
    md11.setObjective(
        t, GRB.MINIMIZE
    )
    
    md11.addConstrs(
        quicksum(l[k] * X[k][i] for k in K) <= t * X[r][i]
        for i in I
    )
    
    md11.addConstrs(
        quicksum(l[k] * Y[k][j] for k in K) >= Y[r][j]
        for j in J
    )
        
#     md11.write(f'md11_{r}.lp')
    md11.params.LogToConsole = 0
    md11.optimize() 
    
    print(df.loc[r,"DMU"])
    for var in md11.getVars():
        print(f"{var.VarName}, {var.X}")

    for k in K:
        crs_efficiency[r] = md11.getVarByName("t").X

AirTran Airways
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.7323961030953262
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
t, 0.9024815523914681
Alaska Airlines
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.35367187586797055
l[8], 0.00038728930051792264
l[9], 0.0
l[10], 0.12725139687488934
l[11], 0.0
l[12], 0.0
t, 0.8333689798418971
American Airlines
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 1.5060913143702905
l[11], 0.22998861631581194
l[12], 0.0
t, 0.9035706345305946
American Eagle Airlines
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.13717360956467017
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.08761019392977855
l[11], 0.0
l[12], 0.0
t, 0.4145731444322367
Continental
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 1.460460973421618
l[8], 0.0
l[9], 0.0
l[10], 0.5186427382281144
l[11], 0.0
l[12], 0.0
t, 

In [21]:
vrs_efficiency = [0 for r in K]

for r in K:
    md12 = Model('model1-2')
    l = md12.addVars(K, vtype=GRB.CONTINUOUS, lb=0, name='l')
    t = md12.addVar(vtype=GRB.CONTINUOUS, name='t')
    
    md12.setObjective(
        t, GRB.MINIMIZE
    )
    
    md12.addConstrs(
        quicksum(l[k] * X[k][i] for k in K) <= t * X[r][i]
        for i in I
    )
    
    md12.addConstrs(
        quicksum(l[k] * Y[k][j] for k in K) >= Y[r][j]
        for j in J
    )
    
    md12.addConstr(quicksum(l[k] for k in K) == 1)
    
    md12.params.LogToConsole = 0
    md12.optimize() 
    
    print(df.loc[r,"DMU"])
    for var in md12.getVars():
        print(f"{var.VarName}, {var.X}")

    for k in K:
        vrs_efficiency[r] = md12.getVarByName("t").X

AirTran Airways
l[0], 1.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
t, 1.0
Alaska Airlines
l[0], 0.0
l[1], 1.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
t, 1.0
American Airlines
l[0], 0.0
l[1], 0.0
l[2], 1.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
t, 1.0
American Eagle Airlines
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 1.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
t, 0.7347929360735309
Continental
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.00876488095238113
l[8], 0.0
l[9], 0.0
l[10], 0.991235119047619
l[11], 0.0
l[12], 0.0
t, 0.8482705912385875
Delta Air Lines
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 1.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
t, 1.

In [230]:
scale_efficiency = [crs_efficiency[k] / vrs_efficiency[k] for k in K]

In [231]:
efficiency_scores = pd.DataFrame(
    [crs_efficiency, vrs_efficiency, scale_efficiency], 
    columns=df["DMU"],
    index=["OE","TE","SE"]
)
efficiency_scores.T

,OE,TE,SE
DMU,,,
AirTran Airways,0.902482,1.000000,0.902482
Alaska Airlines,0.833369,1.000000,0.833369
American Airlines,0.903571,1.000000,0.903571
American Eagle Airlines,0.414573,0.734793,0.564204
Continental,0.808507,0.848271,0.953124
Delta Air Lines,0.964184,1.000000,0.964184
ExpressJet airlines,1.000000,1.000000,1.000000
JetBlue Airways,1.000000,1.000000,1.000000
Northwest Airlines,1.000000,1.000000,1.000000


### iii.(15%) Estimate profit efficiency, and allocative efficiency.

**Parameters** <br/>
$P^x_i$ Cost of input $i$ for DMU $r$. <br/>
$P^y_j$ Revenue of output $j$ for DMU $r$.<br />
**Profit efficiency formulation** <br />
$$ 
\begin{split}
\text{PE} = \min \limits_{\lambda} \quad & P^y_{j} y_{j} - P^x_{i} x_{i} \\
\text{s.t.} \quad
& \sum_{k \in K} \lambda_k X_{ki} \leq x_{ri}  \quad \forall i \in I \\
& \sum_{k \in K} \lambda_k Y_{kj} \geq y_{rj}  \quad \forall j \in J \\
& \sum_{k \in K} \lambda_k = 1 \\
& \lambda_k, x_{i}, y_j \geq 0 \quad \forall k \in K, \forall i \in I, \forall j \in J \\
\end{split}
$$

**Directional Distance Function(Färe, R., Grosskopf, S., & Weber, W. L. ,2004)** <br />

$$ 
\begin{split}
\text{TE} =  \min \limits_{\lambda} \quad & \theta_r \\
\text{s.t.} \quad
& \sum_{k \in K} \lambda_k X_{ki} \leq (1 - \theta_r) X_{ri}  \quad \forall i \in I \\
& \sum_{k \in K} \lambda_k Y_{kj} \geq  (1 + \theta_r) Y_{rj}  \quad \forall j \in J \\
& \sum_{k \in K} \lambda_k = 1 \\
& \lambda_k \geq 0 \quad \forall k \in K \\
& \theta_r \geq 0.
\end{split}
$$
**Allocative efficiency(AE) = PE / (1 - TE)**

In [232]:
cost = [
    list(df["Prce Per Gallons"].values),
    list(df["Price per unit"].values)
]
cost = np.array(cost).T.tolist()
revenue = [
    list(df["Price per Passenger-miles"].values),
    list(df["Price per Ton-miles"].values)
]
rev = np.array(revenue).T.tolist()

In [329]:
best_profit = [0 for r in K]

for r in K:
    md2 = Model('model2')
    x = md2.addVars(I, vtype=GRB.CONTINUOUS, lb=0, name='x')
    y = md2.addVars(J, vtype=GRB.CONTINUOUS, lb=0, name='y')
    l = md2.addVars(K, vtype=GRB.CONTINUOUS, lb=0, name='l')
    
    md2.setObjective(
        quicksum(rev[r][j] * y[j] for j in J) - quicksum(cost[r][i] * x[i] for i in I), 
        GRB.MAXIMIZE
    )
    
    md2.addConstrs(
        quicksum(l[k] * X[k][i] for k in K) <= x[i]
        for i in I
    )
    
    md2.addConstrs(
        quicksum(l[k] * Y[k][j] for k in K) >= y[j]
        for j in J
    )
    
    md2.addConstr(quicksum(l[k] for k in K) == 1)
    
    md2.params.LogToConsole = 0
    md2.optimize() 
    
    for var in md2.getVars():
        print(f"{var.VarName}, {var.X}")

    best_profit[r] = md2.objVal

x[0], 2673000000.0
x[1], 70923.0
y[0], 163483000000.0
y[1], 12449000000.0
l[0], 0.0
l[1], 0.0
l[2], 1.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
x[0], 2673000000.0
x[1], 70923.0
y[0], 163483000000.0
y[1], 12449000000.0
l[0], 0.0
l[1], 0.0
l[2], 1.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
x[0], 2673000000.0
x[1], 70923.0
y[0], 163483000000.0
y[1], 12449000000.0
l[0], 0.0
l[1], 0.0
l[2], 1.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
x[0], 2673000000.0
x[1], 70923.0
y[0], 163483000000.0
y[1], 12449000000.0
l[0], 0.0
l[1], 0.0
l[2], 1.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
x[0], 2673000000.0
x[1], 70923.0
y[0], 163483000000.0
y[1], 12449000000.0
l[0], 0.0
l[1], 0.0
l[2], 1.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.

In [217]:
best_profit

[15993443669.3,
 31057853506.200005,
 14044038678.400005,
 40201318678.4,
 15611091980.400005,
 11893642405.0,
 21976311205.700005,
 14098257861.0,
 12246749028.900002,
 30623221690.1,
 20975889700.700005,
 12643593866.900002,
 14234328358.599998]

In [222]:
profit_efficiency = []
for r in K:
    profit_efficiency.append( 
        np.matmul(np.array(rev[r]), np.array(np.array(Y[r])))
        / best_profit
    )

In [223]:
profit_efficiency

[0.22355910796517606,
 0.1860225143648984,
 2.0108667205135737,
 0.08282987000098395,
 1.0004296957322665,
 1.777863271819126,
 0.09815478038191038,
 0.33830562946319204,
 1.1282243122149653,
 0.11694034142585688,
 0.9952326360346628,
 1.773286949582887,
 0.8203843346739079]

In [379]:
technical_efficiency = [0 for r in K]

for r in K:
    md32 = Model('model3-2')
    l = md32.addVars(K, vtype=GRB.CONTINUOUS, lb=0, name='l')
    t = md32.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1, name='t')
    
    md32.setObjective(
        t, GRB.MAXIMIZE
    )
    
    md32.addConstrs(
        quicksum(l[k] * X[k][i] for k in K) <= X[r][i] * (1 - t)
        for i in I
    )
    
    md32.addConstrs(
        quicksum(l[k] * Y[k][j] for k in K) >= Y[r][j] * (1 + t)
        for j in J
    )
    
    md32.addConstr(quicksum(l[k] for k in K) == 1)
    
    md32.params.LogToConsole = 0
    md32.optimize() 
    
#     for var in md12.getVars():
#         print(f"{var.VarName}, {var.X}")

    technical_efficiency[r] = md32.getVarByName("t").X

In [380]:
technical_efficiency

[0.0,
 0.0,
 0.0,
 0.2358630725503464,
 0.07378746132593614,
 0.0,
 0.0,
 0.0,
 0.0,
 0.09765379593096646,
 0.0,
 0.0,
 0.12246980834370601]

In [381]:
allocate_efficiency = [profit_efficiency[k] / (1 - technical_efficiency[k]) for k in K]
te = [(1 - technical_efficiency[k]) for k in K]

In [382]:
efficiency_scores = pd.DataFrame(
    [profit_efficiency, te, allocate_efficiency], 
    columns=df["DMU"],
    index=["Profit Eff","Technical Eff","Allocative Eff"]
)
efficiency_scores.T

,Profit Eff,Technical Eff,Allocative Eff
DMU,,,
AirTran Airways,0.223559,1.000000,0.223559
Alaska Airlines,0.186023,1.000000,0.186023
American Airlines,2.010867,1.000000,2.010867
American Eagle Airlines,0.082830,0.764137,0.108397
Continental,1.000430,0.926213,1.080130
Delta Air Lines,1.777863,1.000000,1.777863
ExpressJet airlines,0.098155,1.000000,0.098155
JetBlue Airways,0.338306,1.000000,0.338306
Northwest Airlines,1.128224,1.000000,1.128224


### iv.(15%) Estimate cross efficiency of each firm.
**cross efficiency** <br />
$$ 
\begin{split}
\theta^{Cross}_r = \max & \quad \sum_{j \in J} u_jY_{rj} \\
\text{s.t.} \quad
& \sum_{i \in I} v_i X_{ri} = 1 \\
& \sum_{j \in J} u_j Y_{kj} - \sum_{i \in I} v_i X_{ki} \leq 0  \quad \forall k \in K \\
& u_j, v_i \geq 0 \quad \forall j \in J, \forall i \in I
\end{split}
$$

In [338]:
# 存取權重的矩陣
u_weights = [
    [0, 0] for k in K
]
v_weights = [
    [0, 0] for k in K
]

for r in K:
    md2 = Model('model2')
    u = md2.addVars(J, vtype=GRB.CONTINUOUS, name='u')
    v = md2.addVars(I, vtype=GRB.CONTINUOUS, name='v')
    
    md2.setObjective(
        quicksum(u[j] * Y[r][j] for j in J), 
        GRB.MAXIMIZE
    )

    md2.addConstr(
        quicksum(v[i] * X[r][i] for i in I) == 1
    )
    
    md2.addConstrs(
        quicksum(u[j] * Y[k][j] for j in J)        
        - quicksum(v[i] * X[k][i] for i in I) <= 0
        for k in K
    )
    
    md2.params.logtoconsole = 0
    md2.optimize() 
    u_sol = md2.getAttr('X',u)
    v_sol = md2.getAttr('X',v)

    for j in J:
        u_weights[r][j] = u_sol[(j)]
    for i in I:
        v_weights[r][i] = v_sol[(i)]

In [360]:
cross_efficiency = [0 for r in K]
for r in K:
    theta_r = (
        1/len(K) * (
            np.sum(np.matmul(np.array(u_weights), np.array(Y[r])) / np.matmul(np.array(v_weights), np.array(X[r])))
        )
    )
    cross_efficiency[r] = theta_r

In [361]:
cross_efficiency

[0.9694685147580839,
 1.4944825218578903,
 1.9173673452249624,
 0.7483848907553668,
 0.9967073105739394,
 1.8532748557461567,
 2.883850435162967,
 1.3679057620066841,
 1.5731106002627593,
 0.8313887101870714,
 2.234931883489943,
 1.8088701492174508,
 1.3682556101369743]

### v.(15%) Estimate super efficiency of each firm.
$$ 
\begin{split}
\min \limits_{\lambda} \quad & \theta_r^{super} \\
\text{s.t.} \quad
& \sum_{k \in K/\{r\}} \lambda_k X_{ki} \leq \theta_r X_{ri}  \quad \forall i \in I \\
& \sum_{k \in K/\{r\}} \lambda_k Y_{kj} \geq  Y_{rj}  \quad \forall j \in J \\
& \lambda_k \geq 0 \quad \forall k \in K \\
& \theta_r^{super} \text{ is free}.
\end{split}
$$

In [289]:
# 存取權重的矩陣
super_efficiency = [0 for r in K]

for r in K:
    md4 = Model('model4')
    l = md4.addVars(K, vtype=GRB.CONTINUOUS, lb=0, name='l')
    t = md4.addVar(vtype=GRB.CONTINUOUS, name='t')
    
    md4.setObjective(
        t, GRB.MINIMIZE
    )
    
    excluding_set = list(set(K) - set([r]))

    md4.addConstrs(
        quicksum(l[k] * X[k][i] for k in excluding_set) <= t * X[r][i]
        for i in I
    )
    
    md4.addConstrs(
        quicksum(l[k] * Y[k][j] for k in excluding_set) >= Y[r][j]
        for j in J
    )
        

    md4.params.logtoconsole = 0
    md4.optimize() 
    
    for var in md4.getVars():
        print(f"{var.VarName}, {var.X}")

    for k in K:
        super_efficiency[r] = md4.getVarByName("t").X
        
        

l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.7323961030953262
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
t, 0.9024815523914681
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.35367187586797066
l[8], 0.00038728930051789483
l[9], 0.0
l[10], 0.12725139687488932
l[11], 0.0
l[12], 0.0
t, 0.8333689798418968
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 1.5060913143702905
l[11], 0.22998861631581194
l[12], 0.0
t, 0.9035706345305946
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.13717360956467006
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.08761019392977856
l[11], 0.0
l[12], 0.0
t, 0.4145731444322367
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 1.4604609734216178
l[8], 0.0
l[9], 0.0
l[10], 0.5186427382281144
l[11], 0.0
l[12], 0.0
t, 0.8085070511509429
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 

In [290]:
super_efficiency

[0.9024815523914681,
 0.8333689798418968,
 0.9035706345305946,
 0.4145731444322367,
 0.8085070511509429,
 0.9641840108451818,
 1.9481250418188873,
 1.1083149045840583,
 1.0625085354888664,
 0.5298490971876374,
 1.1717749938157043,
 1.0283155469842096,
 0.7558657435490188]

### vi. (10%) Estimate slack-based measure (SBM) of each firm.

**slack-based measure formulation** <br />
$$ 
\theta^{SBM} = \min \limits_{s_i^+,s_i^-} \frac{ 1-\frac{1}{|I|} \sum_{i \in I} \frac{s_i^-}{X_{ri}}}{1+\frac{1}{|J|} \sum_{j \in J} \frac{s_i^+}{Y_{rj}}} \\
\text{s.t.} 
\sum_{k \in K} \lambda_k X_{ki} + s_i^- = X_{ri}  \quad \forall i \in I \\
\sum_{k \in K} \lambda_k Y_{kj} + s_j^+ = Y_{rj}  \quad \forall j \in J \\
\lambda_k \geq 0 \quad \forall k \in K \\
s_i^- \geq 0 \quad \forall i \in I \\
s_j^+ \geq 0 \quad \forall j \in J \\
\lambda \geq 0 \quad \forall k \in K
$$
- SBM can be transformed into a linear programusing the Charnes±Cooper transformation in thesimilar way as the CCR model. (Charnes andCooper, 1962; Charnes et al., 1978.)
- Let us multiply a scalar variable $ (t>0) $to both the denominator and the numerator. 
- This causes no change in objective function. We adjust $t$ so that the denominator becomes 1. Then this term is moved to constraints. The objective is to minimize thenumerator. 

**Linearized SBM-formulation(Kaoru Tone, 2011)** <br />
$$ 
\min \limits_{s_i^+,s_i^-} t-\frac{1}{|I|} \sum_{i \in I} \frac{t s_i^-}{X_{ri}} \\
\text{s.t.} \\
t+\frac{1}{|J|} \sum_{j \in J} \frac{t s_j^+}{Y_{rj}} = 1 \\
\sum_{k \in K} \lambda_k X_{ki} + s_i^- = X_{ri}  \quad \forall i \in I \\
\sum_{k \in K} \lambda_k Y_{kj} - s_j^+ = Y_{rj}  \quad \forall j \in J \\
\lambda_k \geq 0 \quad \forall k \in K \\
s_i^- \geq 0 \quad \forall i \in I \\
s_j^+ \geq 0 \quad \forall j \in J \\
\lambda \geq 0 \quad \forall k \in K
$$
- The problem given above is a nonlinear programming problem since it contains the nonlinearterm $ts_i^+, ts_i^-$. However we can transform it into a linear program as follows. 
- Let $ts_i^+ = S_i^+, ts_j^- = S_j^-, t\lambda = \Lambda$
$$ 
\min \limits_{S_i^+,S_i^-} t-\frac{1}{|I|} \sum_{i \in I} \frac{S_i^-}{X_{ri}} \\
\text{s.t.}  \quad
t+\frac{1}{|J|} \sum_{j \in J} \frac{S_j^+}{Y_{rj}} = 1 \\
\sum_{k \in K} \Lambda_k X_{ki} + S_i^- = t X_{ri}  \quad \forall i \in I \\
\sum_{k \in K} \Lambda_k Y_{kj} - S_j^+ = t Y_{rj}  \quad \forall j \in J \\
\lambda_k \geq 0 \quad \forall k \in K \\
S_i^- \geq 0 \quad \forall i \in I \\
S_j^+ \geq 0 \quad \forall j \in J \\
\Lambda \geq 0 \quad \forall k \in K
$$

In [377]:
sbm_efficiency = [0 for r in K]

for r in K:
    md5 = Model('model5')
    sx = md5.addVars(I, vtype=GRB.CONTINUOUS, lb=0, name='sx')
    sy = md5.addVars(J, vtype=GRB.CONTINUOUS, lb=0, name='sy')
    l = md5.addVars(K, vtype=GRB.CONTINUOUS, lb=0, name='l')
    t = md5.addVar(vtype=GRB.CONTINUOUS, lb=0, name='t')
    
    md5.setObjective(
        t - 1/len(I) * quicksum(sx[i] / X[r][i] for i in I)
        , GRB.MINIMIZE
    )
    
    md5.addConstr(
        (t + 1/len(J) * quicksum(sy[j] / Y[r][j] for j in J) == 1)
    )
    
    md5.addConstrs(
        quicksum(l[k] * X[k][i] for k in K) + sx[i] == t * X[r][i]
        for i in I
    )
    
    md5.addConstrs(
        quicksum(l[k] * Y[k][j] for k in K) - sy[j] == t * Y[r][j]
        for j in J
    )
    
    md5.params.logtoconsole = 0
    md5.optimize() 

    
    for var in md5.getVars():
        print(f"{var.VarName}, {var.X}")
    

    sbm_efficiency[r] = md5.objVal


sx[0], 72325987.65305299
sx[1], 0.0
sy[0], 0.0
sy[1], 497625287.3720191
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.006209666856957031
l[8], 0.0
l[9], 0.0
l[10], 0.16821799465531884
l[11], 0.0
l[12], 0.0
t, 0.7140084555333224
sx[0], 78613071.09067649
sx[1], 1111.6460833644196
sy[0], 0.0
sy[1], 222367307.67206228
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.22285639742853286
l[11], 0.0
l[12], 0.0
t, 0.9181871568535459
sx[0], 350158392.90906096
sx[1], 5096.919316624648
sy[0], 22824253307.357803
sy[1], 0.0
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 1.7553407067092106
l[11], 0.0
l[12], 0.0
t, 0.9301938020853612
sx[0], 0.0
sx[1], 629.8943576079055
sy[0], 4863632997.837175
sy[1], 389950434.43317264
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.08861788531893737
l[11], 0.

In [378]:
sbm_efficiency

[0.6217559202615711,
 0.7572905421265307,
 0.8287618965245473,
 0.349913812007095,
 0.5800144064460465,
 0.9119180909735802,
 1.0,
 1.0,
 1.0,
 0.4534492190684388,
 1.0,
 1.0,
 0.6894336247269977]

### vii.(20%) Based on efficiency analysis you provide above, what managerial implication or insight do you observe/get? How to drive productivity in practice?

In [383]:
efficiency_scores = pd.DataFrame(
    [crs_efficiency, vrs_efficiency, scale_efficiency, profit_efficiency, allocate_efficiency, cross_efficiency, super_efficiency, sbm_efficiency], 
    columns=df["DMU"],
    index=["Overall","Technical","Scale","Profit","Allocate","Cross","Super","SBM"]
)
efficiency_scores.T

,Overall,Technical,Scale,Profit,Allocate,Cross,Super,SBM
DMU,,,,,,,,
AirTran Airways,0.902482,1.000000,0.902482,0.223559,0.223559,0.969469,0.902482,0.621756
Alaska Airlines,0.833369,1.000000,0.833369,0.186023,0.186023,1.494483,0.833369,0.757291
American Airlines,0.903571,1.000000,0.903571,2.010867,2.010867,1.917367,0.903571,0.828762
American Eagle Airlines,0.414573,0.946245,0.564204,0.082830,0.108397,0.748385,0.414573,0.349914
Continental,0.808507,1.000000,0.953124,1.000430,1.080130,0.996707,0.808507,0.580014
Delta Air Lines,0.964184,1.000000,0.964184,1.777863,1.777863,1.853275,0.964184,0.911918
ExpressJet airlines,1.000000,1.000000,1.000000,0.098155,0.098155,2.883850,1.948125,1.000000
JetBlue Airways,1.000000,1.000000,1.000000,0.338306,0.338306,1.367906,1.108315,1.000000
Northwest Airlines,1.000000,1.000000,1.000000,1.128224,1.128224,1.573111,1.062509,1.000000


According to the table above, we can get some insights. <br/>
**DMUs with the best overall effieicny: ExpressJet airlines, JetBlue Airways, Northwest Airlines, Southwest Airline, United Airlines**
1. Most DMUs are technically efficient, only that US Airways has obviously worse technical effieicny than others. It should learn JetBlue and SkyWest since these 2 DMUs form the frontier for US Airways.
2. American Eagle Airlines and SkyWest Airlines have the worst scale efficiencies. Since $\sum_{k \in K}\lambda_k < 1$ under CRS model, we know these two DMUs are increasing returns to scale (IRS). Thus, they should consider expanding the scale of production to achieve scale efficient.
3. ExpressJet airlines has the worst profit efficiency, meaning that from the perspective of profit maximization, it should change the allocation of inputs and outputs to get higher profit.
4. SBM efficiency shows similar insights as overall efficiency. The DMUs on the frontier are the same.
5. Cross efficiency includes peer evaluation, so it can eliminate unrealistic weight schemes. ExpressJet aitlines has the best cross efficiency.
6. Super efficiency is to do sensitivity test on all DMUs. It shows that ExpressJet airlines, JetBlue Airways, Northwest Airlines, Southwest Airline, United Airlines are still frontier DMUs. It shows that the DEA is stable.